In [35]:
import csv
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
from sklearn import tree

In [36]:
def findCommunities():
    communities = []
    with open('C:\\Users\\ar1\\Documents\ML\\Project\\Crime Prediction Data(1)\\Crime Prediction Data\\communities-crime-clean.csv', 'r') as csvfile:
        filereader = csv.reader(csvfile)
        j = 0
        for i in filereader:
            if j > 0:
                if i[1] not in communities:
                    communities.append(i[1])
            j+=1
    return communities


In [86]:
def extractData(filename,communities,addstate = False,addcommunity = False):
    features = []
    x=[]
    y=[]
    with open(filename, 'r') as csvfile:
        filereader = csv.reader(csvfile)
        j = 0
        for i in filereader:
            if j > 0:
                data = i[3:len(i)-1]
                if addstate:
                    state = [0.0]*56
                    state[int(i[0])-1] = 1.0
                    data = data + state
                    features = features + ["state"+str(i) for i in range(56)]
                if addcommunity:
                    tmp = [0.0]*len(communities)
                    tmp[communities.index(i[1])] = 1.0
                    data = data + tmp
                    features = features + communities
                vect = np.array(data)
                #vect = np.array(i[3:len(i)-1])
                x.append(vect.astype(np.float))
                if float(i[-1]) > 0.1:
                    y.append(1)
                else:
                    y.append(0)
            else:
                features = i[3:len(i)-1]
            j+=1
        y = np.array(y)
        x = np.array(x)
        features = np.array(features)
        return x,y,features


In [87]:
def calcualtePercentage(labels):
    n = len(labels)
    sumTrue = sum(labels)*1.0
    sumFalse = (n - sumTrue)*1.0
    return sumTrue/n,sumFalse/n


In [88]:
# Gaussian. To be put in a seperate file
def gaussianNB():
    return GaussianNB()


In [89]:
def linearSVC():
    return LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=1.0, multi_class='ovr', 
                     fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=None,
                     max_iter=1000)

In [90]:

def confusionMatrix(true,predicted,labels = [1,0]):
    return confusion_matrix(true,predicted,labels)


In [91]:

def find_confusion_matrix(actual,predicted,clabels):
    cm= []
    for i in clabels:
        tmp =[0]*len(clabels)
        
        for j in range(len(actual)):
            if actual[j]== i and actual[j] == predicted[j]:
                tmp[clabels.index(i)] += 1
            elif actual[j]== i and actual[j] != predicted[j]:
                tmp[clabels.index(predicted[j])] += 1
        cm.append(tmp)
    return np.array(cm)


In [92]:
def find_accuracy(matrix):
    return np.trace(matrix)*1.0/np.sum(matrix)


In [93]:
def find_precision(matrix):
    pres = []
    x = np.sum(matrix,axis=0)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                pres.append(matrix[i][j]*1.0/x[i])
    return pres

In [94]:
def find_recall(matrix):
    rec = []
    x = np.sum(matrix,axis=1)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                rec.append(matrix[i][j]*1.0/x[i])
    return rec

In [95]:
def find_fmeasure(prec,rec):
    tmp = []
    for i,j in zip(prec,rec):
        tmp.append(2.0*(i*j)/(i+j))
    return tmp

In [96]:
def evaluation(acutal,predicted,clabels=[1,0]):
    confmatrix = find_confusion_matrix(acutal,predicted,clabels)
    print ("Confusion Matrix")
    print (confmatrix)
    accuracy = find_accuracy(confmatrix)
    print ("Accuracy", accuracy)
    precision = find_precision(confmatrix)
    print ("Precision", precision)
    recall = find_recall(confmatrix)
    print ("Recall", recall)
    f_score =find_fmeasure(precision,recall)
    print ("F_score", f_score)

In [97]:
def do_cross_validation(X, y,clf, n_folds=5):
    cv = KFold(len(y), n_folds)
    accuracies = []
    for train_ind, test_ind in cv: 
        clf.fit(X[train_ind], y[train_ind])
        predictions = clf.predict(X[test_ind])
        evaluation(y[test_ind],predictions)
        accuracies.append(accuracy_score(y[test_ind], predictions))
    avg = np.mean(accuracies)
    return avg

In [98]:
distintCommunities = findCommunities()

In [99]:
filename= 'C:\\Users\\ar1\\Documents\ML\\Project\\Crime Prediction Data(1)\\Crime Prediction Data\\communities-crime-clean.csv'
X,Y,Features = extractData(filename, distintCommunities)

In [100]:
calcualtePercentage(Y)

(0.62719518314099343, 0.37280481685900652)

In [101]:
svc= linearSVC()
svc.fit(X[:1000],Y[:1000])
predictedY=svc.predict(X[1000:])

In [118]:
topFeature = np.argsort(svc.coef_)[0][::-1][:10]
print (topFeature)
for i in topFeature:
    print (Features[i],svc.coef_[0][i])


[68 94 60 91  6 38 32 39 27 52]
PctPersDenseHous 0.984908412363
PctSameCity85 0.981435176761
PctSpeakEnglOnly 0.960729845933
PctForeignBorn 0.914924461111
agePct12t21 0.914760315051
MalePctDivorce 0.912014512711
PctUnemployed 0.868226310952
MalePctNevMarr 0.84859120791
NumUnderPov 0.746730894897
PctImmigRecent 0.711115244969


Explaination : 10 most predictive feature
2-b-ii) The 10 most predicitve features can be found above. 

In [119]:
confusionMatrix(Y[1000:],predictedY)

array([[447,  94],
       [126, 326]])

In [120]:
evaluation(Y[1000:],predictedY)

Confusion Matrix
[[447  94]
 [126 326]]
Accuracy 0.778449144008
Precision [0.78010471204188481, 0.77619047619047621]
Recall [0.82624768946395566, 0.72123893805309736]
F_score [0.80251346499102338, 0.74770642201834858]


In [121]:
evaluation(Y[1000:],predictedY)

Confusion Matrix
[[447  94]
 [126 326]]
Accuracy 0.778449144008
Precision [0.78010471204188481, 0.77619047619047621]
Recall [0.82624768946395566, 0.72123893805309736]
F_score [0.80251346499102338, 0.74770642201834858]


In [125]:
do_cross_validation(X[:1000],Y[:1000],svc,10)

Confusion Matrix
[[68  6]
 [10 16]]
Accuracy 0.84
Precision [0.87179487179487181, 0.72727272727272729]
Recall [0.91891891891891897, 0.61538461538461542]
F_score [0.89473684210526316, 0.66666666666666674]
Confusion Matrix
[[88  3]
 [ 6  3]]
Accuracy 0.91
Precision [0.93617021276595747, 0.5]
Recall [0.96703296703296704, 0.33333333333333331]
F_score [0.95135135135135129, 0.40000000000000002]
Confusion Matrix
[[88  2]
 [ 3  7]]
Accuracy 0.95
Precision [0.96703296703296704, 0.77777777777777779]
Recall [0.97777777777777775, 0.69999999999999996]
F_score [0.97237569060773477, 0.73684210526315774]
Confusion Matrix
[[77  5]
 [11  7]]
Accuracy 0.84
Precision [0.875, 0.58333333333333337]
Recall [0.93902439024390238, 0.3888888888888889]
F_score [0.90588235294117636, 0.46666666666666662]
Confusion Matrix
[[51  1]
 [10 38]]
Accuracy 0.89
Precision [0.83606557377049184, 0.97435897435897434]
Recall [0.98076923076923073, 0.79166666666666663]
F_score [0.90265486725663724, 0.87356321839080453]
Confusion M

0.82899999999999996

Explaination: Cross Validation
2-b-i) The 10 fold cross validation accuracy,precision,recall and F_score can be found above. 

In [127]:
accuracy_score(Y[1000:],predictedY)

0.77844914400805643

Explaination : Accuracy compared with decision tree
2-b-iii) Accuracy for decision tree was 72% where as the accuracy for Linear SVC is 77.8 %. Thus it can be concluded that Linear SVC was able to predict much better than Decision Tree. 

In [128]:
sumCorrect =0
for i in range(1000,len(Y)):
    if Y[i]==predictedY[i-1000]:
        sumCorrect += 1
sumCorrect*1.0/len(Y[1000:])

0.7784491440080564

In [ ]:
-